# Stock Prediction

The following notebook was an assignment for the intro computer science class at Rice. I'm writing it up in a jupyter notebook for a few reasons: First off, I want to practice using Github. Secondly, I wanted to practice using 
Jupyter notebooks to make projects more comprehensible (as compared to IDEs). Finally, I thought it wise to go over past projects and improve upon them - e.g. by implementing TA feedback.

### Note: unfortunately, this project relies on a module that is unavailable outside of CodeSkulptor. However, the results are included.

In [3]:
"""
Stock market prediction using Markov chains.
"""
# comp140_module3 is unavailable outside of CodeSkulptor, the environment we coded in 
# for this class
#import comp140_module3 as stocks
import random
import math


### The following is an excerpt from the project description:


We will use a Markov chain as a statistical model of a stock's performance. We will determine the likelihood that a stock will go up or down by building this model from the history of the stock's behavior. Modeling stock market performance is a difficult and important problem. We will only be scratching the surface of what is possible. While we will be able to get reasonably accurate predictions, you should not run off and start investing your money based on the results of this project!

It is quite difficult to predict the exact price fluctuations of a stock from day to day. Instead we will try to predict how large the change will be. Given $P_i$, the price on day $i$, and $P_i+1$, the price on day $i+1$, we will consider $ \delta_i+1= (P_i+P_{i+1})\ / \ P_i $ to be the change in price from day $i$ to day $i+1$. Using our model, we will try to predict in which of the following four ranges $\delta_i+1$ will lie:

$$\begin{cases}(-\infty, -0.01\ \text{(larger than 1% decrease)}\\
[-0.01,0)\\
[0,0.01)\\
[0.01,\infty)\ \text{ (larger than 1% increase)}\end{cases}$$

We will represent these four "bins" with the numbers $0$ through $3$, respectively, as indicated in the list above. I.e., bin $2$ is $[0,0.01)$.

To make this prediction, we will construct an $n^{th}$ order Markov chain using the history of these changes for $2$ years for each of two stocks, Google (GOOG) and First Solar Inc. (FSLR), and a stock index, the Dow Jones Industrial Average (DJIA).

## Part 1: Creating the model

In this section, we create our model, given historical data. $$\\ $$ Specifically, the chain is implemented as a nested dictionary; the outer keys are tuples with the $n$ past states (where $n$ is the order of the chain). The inner dictionary maps bins to their respective transition probabilities.


In [ ]:
### helper func

def probability_dict_from(bin_count_dict):
    """
    bin_count_dict : a dictionary mapping 1-4 of the
        four possible bins to the number of times the bin
        occurs after a specific memory sequence
    
    This is a helper function to create the nth order markov
    chain. It takes in the dictionary described above and 
    maps each bin to probabilities instead of counts.
    """
    total = sum(bin_count_dict.values())
    for bag in bin_count_dict:
        count = bin_count_dict[bag]
        bin_count_dict[bag] = float(count) / total
    return bin_count_dict

## Testing

#test_dict = {1: 20, 2: 80}
#print(probability_dict_from(test_dict))


In [15]:
### Model

def markov_chain(data, order):
    """
    Create a Markov chain with the given order from the 
    given list of data. Returns a nested dictionary of form:
    {(past_n_states): {bin_x: # times bin_x follows past_n_states,...}, ...}
    
    data   - a list of numbers, where each number is a bin (0 to 3)
    order  - order of the desired markov chain
    """
    chain = {}  
    length = len(data)
    
    # this for loop creates the nested dict
    # ranges to "length - order" so no index error
    for start in range(0, length - order):
        
        # extract past states, currrent state
        end = start + order
        slicey = data[start:end]  # sliding window - doesn't include end
        past_states = tuple(slicey)
        bag = data[end]
        
        # create nested dictionary
        if past_states not in chain:
            chain[past_states] = {}
        if bag not in chain[past_states]:
            chain[past_states][bag] = 0
        chain[past_states][bag] += 1
    
    
    for tup in chain:
        chain[tup] = probability_dict_from(chain[tup])
    return chain

##Testing

#d = [1,2,3,1,2,3,1,2,3,1,2,3]
#print(markov_chain(d, 5))
#bd = [1,0,4,2,3,1,1,2,3,2,2,1,1,1,2,3,3,2,2,4,2,2,3,4,1,2,3,4,2,3,1,2]
#print(markov_chain(bd, 1))

## Part 2: Prediction & error

In this section, we predict the next $m$ states, given an $n^{th}$ order markov chain and the last $n$ states.

If there exists a state that isn't in the markov chain, randomly predict the next day's change (choose randomly among the 4 bins). 

We also write a mean squared error function.

In [ ]:

### Predict

def predict(model, last, num):
    """
    Predicts the next num values given the model and the last values.
    Returns a list of predictions.
    
    model - a model in the nested dictionary form; in the 
            form of a model created from the "markov_chain" func
    last  - last n states
    num   - number of values to predict
    """
    predictions = []
    not_last = list(last)
    
    # this while loop generates the predictions
    while num > 0:
        num -= 1
        
        if tuple(not_last) in model:
            prob_counter0 = 0
            prob_counter1 = 0
            #randomly generate a float f, 0<=f<=1
            bin_selector = random.random()
            #this for loop deals with which
            #transition to take.
            for bag in model[tuple(not_last)]:
                prob_counter1 += model[tuple(not_last)][bag]
                #we're seeing where bin_selector
                #falls in the number line between 0 and 1
                #and selecting the respective bin.
                if bin_selector > prob_counter0 and bin_selector < prob_counter1:
                    prob_counter0 += model[tuple(not_last)][bag]
                    not_last.append(bag)
                    not_last.pop(0)
                    predictions.append(bag)
                    #break so we don't add > 1 bin per transition
                    break
                    
        #randomly select a bin
        else: 
            random_bin = random.randint(0,3) 
            predictions.append(random_bin)
            not_last.pop(0)
            not_last.append(random_bin)
        
            
    return predictions


##Testing
#print(predict({(0,): {0: 0.25, 1: 0.25, 2: 0.5}}, [0], 1))



In [ ]:

### Error

def mse(result, expected):
    """
    Calculate the mean squared error between the sequences 
    result and expected.
    
    result  - list of predicted values
    expected - list of actual values
    """
    length = len(result)
    total_error = 0
    counter = 0
    for element in expected:
        specific_error = (element - result[counter])
        total_error += specific_error ** 2
        counter += 1
    mean_squared_error = (1/float(length)) * total_error
    
    return mean_squared_error

## Testing
#print(mse([1,1,1],[1,1,2]))


## Part 3: Experimentation

In this section, we write a function to predict behavior of a given stock/ index, using the other functions 
we wrote.


In [ ]:

### Experiment

def run_experiment(train, order, test, future, actual, trials):
    """
    Run an experiment to predict the future of the test
    data given the training data.  Returns the average 
    mean squared error over the number of trials.
    
    train  - training data
    order  - order of the markov model to use
    test   - "order" days of testing data
    future - number of days to predict
    actual - actual results for next "future" days
    trials - number of trials to run
    """ 
    total_error = 0
    number = trials
    model = markov_chain(train, order)
    
    while number > 0:
    
        predictions = predict(model, test, future)
    
        specific_error = mse(actual, predictions)
        
        total_error += specific_error
        
        number -= 1
        
    average_mse = float(total_error) / trials
   
    
    return average_mse


In [ ]:
###Application

def run():
    """
    Run application.

    """
    # Get the supported stock symbols
    symbols = stocks.get_supported_symbols()
    
    # Get stock data and process it

    # Training data
    changes = {}
    bins = {}
    for symbol in symbols:
        prices = stocks.get_historical_prices(symbol)
        changes[symbol] = stocks.compute_daily_change(prices)
        bins[symbol] = stocks.bin_daily_changes(changes[symbol])

    # Test data
    testchanges = {}
    testbins = {}
    for symbol in symbols:        
        testprices = stocks.get_test_prices(symbol)
        testchanges[symbol] = stocks.compute_daily_change(testprices)
        testbins[symbol] = stocks.bin_daily_changes(testchanges[symbol])

    # Display data
    stocks.plot_daily_change(changes)
    stocks.plot_bin_histogram(bins)

    # Run experiments
    orders = [1, 3, 5, 7, 9]
    ntrials = 500
    days = 5

    for symbol in symbols:
        print(symbol)
        print("====")
        print("Actual:", testbins[symbol][-days:])
        for order in orders:
            error = run_experiment(bins[symbol], order,
                                   testbins[symbol][-order-days:-days], days, 
                                   testbins[symbol][-days:], ntrials)
            print("Order", order, ":", error)            
 
#run()
